# Data Exploration

In [100]:
from pathlib import Path
import numpy as np
import pandas as pd
import duckdb
import sklearn

In [2]:
# Display Options for Pandas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = None

In [3]:
data_path = Path("../data")
data_path_string = data_path.resolve().as_posix()

### ETL

In [4]:
con = duckdb.connect(database=f"{data_path}/spotify.db")

In [53]:
# Clean DB 
con.execute("drop table if exists lyrics_features")
con.execute("drop table if exists low_level_audio_features")
con.execute("drop table if exists albums")
con.execute("drop table if exists artists")
con.execute("drop table if exists tracks")
con.execute("drop table if exists features")
con.execute("drop view if exists lookup")

In [54]:
# Read data into DB
con.read_csv(f"{data_path_string}/lyrics_features.csv", all_varchar=False).create("lyric_features")
con.read_csv(f"{data_path_string}/low_level_audio_features.csv", all_varchar=False).create("low_level_audio_features")
con.read_csv(f"{data_path_string}/spotify_albums.csv", all_varchar=False).create("albums")
con.read_csv(f"{data_path_string}/spotify_artists.csv", all_varchar=False).create("artists")
con.read_csv(f"{data_path_string}/spotify_tracks.csv", all_varchar=False).create("tracks")

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [55]:
con.table("lyric_features").limit(10).df()

,column0,mean_syllables_word,mean_words_sentence,n_sentences,n_words,sentence_similarity,track_id,vocabulary_wealth
0,0,-1.00,-1.00,-1,-1,-1.000000,5KIfHjHI5NIsPHNt58qua0,-1.00
1,1,1.10,5.65,31,326,0.043011,13keyz9ikBe6ZpRasw7l4X,0.45
2,2,1.37,4.77,74,532,0.050352,1WugzepXsLjnsM0K4UaWYc,0.59
3,3,1.95,3.38,72,430,0.028560,2MO6oEAlMKcsfI8xP3yoy8,0.49
4,4,1.16,2.99,68,368,0.047849,1i4St7fmSUE9nB3R9n8fol,0.47
5,5,1.32,4.21,39,256,0.040486,3UyfvY3Gs6d4wvq8O4ANqQ,0.60
6,6,1.24,2.98,50,287,0.038367,2zhk0kypGeOPsaaZcjyc14,0.49
7,7,1.32,3.44,86,524,0.050068,3pjampEJWz0wJuiFYyzX0x,0.59
8,8,1.16,3.11,38,194,0.083926,3xBGFVU2b2PV556Q3ebsQH,0.58
9,9,1.86,2.94,34,187,0.048128,4JbTcgrkSwBieKJ8GInB02,0.62


In [56]:
con.table("low_level_audio_features").limit(10).df()


,column000,Chroma_1,Chroma_10,Chroma_11,Chroma_12,Chroma_2,Chroma_3,Chroma_4,Chroma_5,Chroma_6,Chroma_7,Chroma_8,Chroma_9,MEL_1,MEL_10,MEL_100,MEL_101,MEL_102,MEL_103,MEL_104,MEL_105,MEL_106,MEL_107,MEL_108,MEL_109,MEL_11,MEL_110,MEL_111,MEL_112,MEL_113,MEL_114,MEL_115,MEL_116,MEL_117,MEL_118,MEL_119,MEL_12,MEL_120,MEL_121,MEL_122,MEL_123,MEL_124,MEL_125,MEL_126,MEL_127,MEL_128,MEL_13,MEL_14,MEL_15,MEL_16,MEL_17,MEL_18,MEL_19,MEL_2,MEL_20,MEL_21,MEL_22,MEL_23,MEL_24,MEL_25,MEL_26,MEL_27,MEL_28,MEL_29,MEL_3,MEL_30,MEL_31,MEL_32,MEL_33,MEL_34,MEL_35,MEL_36,MEL_37,MEL_38,MEL_39,MEL_4,MEL_40,MEL_41,MEL_42,MEL_43,MEL_44,MEL_45,MEL_46,MEL_47,MEL_48,MEL_49,MEL_5,MEL_50,MEL_51,MEL_52,MEL_53,MEL_54,MEL_55,MEL_56,MEL_57,MEL_58,MEL_59,MEL_6,MEL_60,MEL_61,MEL_62,MEL_63,MEL_64,MEL_65,MEL_66,MEL_67,MEL_68,MEL_69,MEL_7,MEL_70,MEL_71,MEL_72,MEL_73,MEL_74,MEL_75,MEL_76,MEL_77,MEL_78,MEL_79,MEL_8,MEL_80,MEL_81,MEL_82,MEL_83,MEL_84,MEL_85,MEL_86,MEL_87,MEL_88,MEL_89,MEL_9,MEL_90,MEL_91,MEL_92,MEL_93,MEL_94,MEL_95,MEL_96,MEL_97,MEL_98,MEL_99,MFCC_1,MFCC_10,MFCC_11,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_2,MFCC_20,MFCC_21,MFCC_22,MFCC_23,MFCC_24,MFCC_25,MFCC_26,MFCC_27,MFCC_28,MFCC_29,MFCC_3,MFCC_30,MFCC_31,MFCC_32,MFCC_33,MFCC_34,MFCC_35,MFCC_36,MFCC_37,MFCC_38,MFCC_39,MFCC_4,MFCC_40,MFCC_41,MFCC_42,MFCC_43,MFCC_44,MFCC_45,MFCC_46,MFCC_47,MFCC_48,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,Spectral_contrast_1,Spectral_contrast_2,Spectral_contrast_3,Spectral_contrast_4,Spectral_contrast_5,Spectral_contrast_6,Spectral_contrast_7,Tonnetz_1,Tonnetz_2,Tonnetz_3,Tonnetz_4,Tonnetz_5,Tonnetz_6,ZCR,entropy_energy,spectral_bandwith,spectral_centroid,spectral_rollOff_max,spectral_rollOff_min,track_id
0,0,0.438296,0.472769,0.427441,0.436688,0.467697,0.493862,0.512244,0.568658,0.560524,0.513068,0.544648,0.544643,0.000140,0.667637,1.630314e-05,8.691743e-06,6.494484e-06,5.488073e-06,4.941570e-06,4.848880e-06,4.890052e-06,4.891686e-06,4.853262e-06,4.997161e-06,1.036885,4.997992e-06,5.193198e-06,5.239410e-06,5.344698e-06,5.402805e-06,5.632341e-06,5.929073e-06,4.468151e-06,2.706399e-07,3.611252e-09,0.960690,3.454723e-09,3.324464e-09,3.249713e-09,3.211787e-09,3.196705e-09,3.123937e-09,3.096601e-09,3.132088e-09,3.115733e-09,1.309046,1.727061,1.172454,2.451283,2.383262,3.495376,1.708026,0.001025,1.812995,2.062649,1.269325,0.964405,0.662044,0.739214,0.527574,0.688505,0.454942,0.311517,0.017772,0.296182,0.268481,0.286426,0.314140,0.346320,0.244440,0.223053,0.363009,0.532202,0.552576,0.292901,0.418687,0.308175,0.248862,0.204990,0.144929,0.106920,0.073315,0.041151,0.037855,0.031073,1.598215,0.026433,0.024971,0.017033,0.011429,0.010716,0.009004,0.009666,0.009330,0.010194,0.008812,2.168316,0.009764,0.010562,0.008871,0.008313,0.006119,0.004583,0.003738,0.005562,0.007731,0.006603,1.188808,0.006090,0.006469,0.004985,0.004448,3.945915e-03,3.418002e-03,2.922538e-03,2.660054e-03,2.991876e-03,2.803772e-03,0.171523,2.464534e-03,2.034050e-03,1.669075e-03,1.571831e-03,1.691151e-03,1.971049e-03,2.169552e-03,1.761862e-03,1.506319e-03,1.307624e-03,0.357523,1.449245e-03,1.438523e-03,1.131707e-03,9.474515e-04,7.292215e-04,4.232844e-04,2.888054e-04,1.518297e-04,7.281402e-05,3.155982e-05,-373.998138,-7.316626,-16.166148,-1.957894,-1.502101,-4.469502,-1.413008,-6.843215,-3.468242,-2.280824,-4.339178,139.977509,-5.973232,-6.111434,-5.065900,-6.364006,-9.749821,-7.584262,-4.942808,-5.604710,-6.234820,-7.207380,-31.957462,-4.700275,-4.880559,-5.913836,-4.999497,-3.399483,-4.613783,-5.075487,-3.299941,-3.443411,-3.432436,11.430949,-2.920989,-2.365429,-2.451597,-1.518548,-1.851851,-0.415478,1.505456,1.401088,1.320392,-1.840450,-14.810051,6.317596,-7.692614,-9.200216,21.827447,13.797794,15.869637,16.190687,16.093940,16.975988,33.174617,0.003370,0.000450,-0.007696,0.018434,-0.001759,-0.006392,0.067966,-89.113389,2564.247669,3558.400706,4508.506071,367.831109,19YEk4OVQZn3GfoxbpNrU6
1,1,0.596605,0.368288,0.285263,0.302211,0.905805,0.510909,0.221708,0.311248,0.491277,0.416469,0.41

In [57]:
con.table("albums").limit(5).df()

,column00,album_type,artist_id,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,track_id,track_name_prev,uri,type
0,0,single,3DiDSECUqqY1AuBP8qtaIa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/1gAM7M4rBwEbSPeAQR2nx1'},https://api.spotify.com/v1/albums/1gAM7M4rBwEbSPeAQR2nx1,1gAM7M4rBwEbSPeAQR2nx1,"[{'height': 640, 'url': 'https://i.scdn.co/image/5872e4d8fac4ef7552576d481b1d676189b4056a', 'wid...",If I Ain't Got You EP,2019-02-08,day,6,2iejTMy9XZ8Gaae0aQ2yl0,track_32,spotify:album:1gAM7M4rBwEbSPeAQR2nx1,album
1,1,album,6s1pCNXcbdtQJlsnM1hRIA,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/4KfJZV7WfolYlxBzOTo66s'},https://api.spotify.com/v1/albums/4KfJZV7WfolYlxBzOTo66s,4KfJZV7WfolYlxBzOTo66s,"[{'height': 640, 'url': 'https://i.scdn.co/image/c75e3ad75b39aed687e365b3bd2e3f7ff603e6a7', 'wid...",Shostakovich Symphony No.5 - Four Romances on Poems by Pushkin,2019-03-01,day,8,1WQfghEjszJJ4H8MAWrQ2C,track_11,spotify:album:4KfJZV7WfolYlxBzOTo66s,album
2,2,single,5YjfNaHq05WrwldRe1QSBc,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/7nLYY7uAVUb57kpd7tZxnS'},https://api.spotify.com/v1/albums/7nLYY7uAVUb57kpd7tZxnS,7nLYY7uAVUb57kpd7tZxnS,"[{'height': 640, 'url': 'https://i.scdn.co/image/b4c9ab9d95bd46d2125c581ca67e8f35bbc052d2', 'wid...",Take My Bass,2019-03-14,day,1,3jJKj4QTK3v18ZSwpk7AcV,track_15,spotify:album:7nLYY7uAVUb57kpd7tZxnS,album
3,3,single,2G9Vc16JCpnZmK4uGH46Fa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/6p20Rt4x2Qn5mUMRi1s6pj'},https://api.spotify.com/v1/albums/6p20Rt4x2Qn5mUMRi1s6pj,6p20Rt4x2Qn5mUMRi1s6pj,"[{'height': 640, 'url': 'https://i.scdn.co/image/4baa2861555a552baf2bcdaacdd7b216fa9b1071', 'wid...",Hypnotizing (Are U),2016-11-16,day,1,1xGtDafUZbHyYC3Xarcbrj,track_46,spotify:album:6p20Rt4x2Qn5mUMRi1s6pj,album
4,4,single,2dwM9OcE4c3Ph1UBINSodx,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/1XeoOqC1q7U2iyLEQJ64cu'},https://api.spotify.com/v1/albums/1XeoOqC1q7U2iyLEQJ64cu,1XeoOqC1q7U2iyLEQJ64cu,"[{'height': 640, 'url': 'https://i.scdn.co/image/f277b7afb683f1c1e30b6737ea7b29053c9ae66d', 'wid...",Sunshine,2018-07-20,day,1,0gWtsXvXOzAT6FtM3ur8in,track_10,spotify:album:1XeoOqC1q7U2iyLEQJ64cu,album


In [58]:
con.table("artists").limit(10).df()

,column0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo tradicional', 'sertanejo universitario']",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
1,1,22,313,[],1dLnVku4VQUOLswwDFvRc9,The Grenadines,4wqwj0gA8qPZKLl5WVqXml,track_30,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,['uk alternative pop'],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,['french baroque'],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist
5,5,43,81,[],38VBjthd0szbS6wpDMafrb,Filhos,453KeZU566kjNfs1IH4yHU,track_15,artist
6,6,34,8358,[],36mHwYa65L0WZbAXY2iSGw,Eloq,6hC5Tl0S5aQCw646JuN91w,track_43,artist
7,7,7,158,[],1jJyy00XfxjB4tMAv5OT0q,Fravær,5005eeCJ9KTFAAzh2NC9mr,track_48,artist
8,8,21,30,[],10A8WbBJ0zW8MnvTsUL6os,Camille Pépin,56fBsCXwpBnxe4JkF66gyF,track_73,artist
9,9,25,992,['classic finnish pop'],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist


In [59]:
con.table("tracks").limit(5).df()

,column00,acousticness,album_id,analysis_url,artists_id,available_markets,country,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,liveness,loudness,lyrics,mode,name,playlist,popularity,preview_url,speechiness,tempo,time_signature,track_href,track_name_prev,track_number,uri,valence,type
0,0,0.294,0D3QufeCudpQANOR7luqdr,https://api.spotify.com/v1/audio-analysis/5qljLQuKnNJf4F4vfxQB0V,['3mxJuHRn2ZWD5OofvJtDZY'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.698,1.0,235584.0,0.606,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,5qljLQuKnNJf4F4vfxQB0V,0.000003,10.0,0.151,-7.447,"\r\n\r\nPerhaps I am bound to be restless\r\nAlways yearning, never satisfied\r\nPerhaps I'm ach...",0.0,Blood,Hipsteribrunssi,28.0,https://p.scdn.co/mp3-preview/1b05a902da3a251d07a38aa710ffae559fc33d08?cid=b3cdb16d0df2409abf6a8...,0.0262,115.018,4.0,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,track_14,1.0,spotify:track:5qljLQuKnNJf4F4vfxQB0V,0.6220,track
1,1,0.863,1bcqsH5UyTBzmh9YizdsBE,https://api.spotify.com/v1/audio-analysis/3VAX2MJdmdqARLSU5hPMpm,['4xWMewm6CYMstu0sPgd9jJ'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.719,1.0,656960.0,0.308,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,3VAX2MJdmdqARLSU5hPMpm,0.000000,6.0,0.253,-10.340,\r\nYour Gods and my Gods-do you or I know which are the stronger? Native Proverb.\r\nEAST of Su...,1.0,The Ugly Duckling,Animal Stories,31.0,https://p.scdn.co/mp3-preview/d8140736a6131cb5595f061975173a272c343a0a?cid=b3cdb16d0df2409abf6a8...,0.9220,115.075,3.0,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,track_3,3.0,spotify:track:3VAX2MJdmdqARLSU5hPMpm,0.5890,track
2,2,0.750,4tKijjmxGClg4JOLAyo2qE,https://api.spotify.com/v1/audio-analysis/1L3YAhsEMrGVvCgDXj2TYn,['3hYaK5FF3YAglCj5HZgBnP'],['GB'],BE,0.466,1.0,492840.0,0.931,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,1L3YAhsEMrGVvCgDXj2TYn,0.000000,4.0,0.938,-13.605,"\r\n\r\nClosed off from love, I didn't need the pain\r\nOnce or twice was enough and it was all ...",0.0,Jimmy Launches His Own Range Of Greetings Cards,Best Of British Comedy,31.0,https://p.scdn.co/mp3-preview/c8af28fb15185b18977152eb50eefef8d90af5a2?cid=b3cdb16d0df2409abf6a8...,0.9440,79.565,4.0,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,track_4,4.0,spotify:track:1L3YAhsEMrGVvCgDXj2TYn,0.0850,track
3,3,0.763,6FeJF5r8roonnKraJxr4oB,https://api.spotify.com/v1/audio-analysis/6aCe9zzoZmCojX7bbgKKtf,['2KQsUB9DRBcJk17JWX1eXD'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.719,1.0,316578.0,0.126,https://api.spotify.com/v1/tracks/6aCe9zzoZmCojX7bbgKKtf,6aCe9zzoZmCojX7bbgKKtf,0.000000,3.0,0.113,-20.254,"\r\nPAIN, n. An uncomfortable frame of mind that may have a physical basis in something that is ...",0.0,The Crime At Pickets Mill,Short Stories,14.0,https://p.scdn.co/mp3-preview/7629b8e9f31f6e9bdfbb11cc8533613340c98cdc?cid=b3cdb16d0df2409abf6a8...,0.9380,112.822,3.0,https://api.spotify.com/v1/tracks/6aCe9zzoZmCojX7bbgKKtf,track_9,1.0,spotify:track:6aCe9zzoZmCojX7bbgKKtf,0.5330,track
4,4,0.770,4tKijjmxGClg4JOLAyo2qE,https://api.spotify.com/v1/audio-analysis/1Vo802A38tPFHmje1h91um,['3hYaK5FF3YAglCj5HZgBnP'],['GB'],BE,0.460,1.0,558880.0,0.942,https://api.spotify.com/v1/tracks/1Vo802A38tPFHmje1h91um,1Vo802A38tPFHmje1h91um,0.000000,7.0,0.917,-13.749,"\r\n\r\nClosed off from love, I didn't need the pain\r\nOnce or twice was enough and it was all ...",1.0,Setup + Punchline = Joke,Best Of British Comedy,32.0,https://p.scdn.co/mp3-preview/32be593c0eb82868dbc18f3c1677722fd6beff8b?cid=b3cdb16d0df2409abf6a8...,0.9430,81.260,4.0,https://api.spotify.com/v1/tracks/1Vo802A38tPFHmje1h91um,track_2,2.0,spotify:track:1Vo802A38tPFHmje1h91um,0.0906,track


In [80]:
# Construct Lookup View
sql = """
drop view if exists lookup;
create view lookup as
select 
    t.id, t.name as track_name, ar.name as artist_name, a.name as album_name, 
    t.preview_url, t.track_href, t.analysis_url
from tracks t
left join albums a on t.album_id = a.id
--left join artists ar on ar.track_id = t.id
left join artists ar on a.artist_id = ar.id
"""
con.execute(sql)

In [61]:
# Construct Features Table
sql = """
create table features as
select
    t.id, t.acousticness, t.danceability, t.duration_ms, t.energy, t.instrumentalness, 
    t.liveness, t.loudness, t.mode, t.popularity, t.speechiness, t.tempo, t.valence,
    lf.mean_syllables_word, lf.mean_words_sentence, lf.n_sentences, lf.n_words, lf.sentence_similarity, lf.vocabulary_wealth
from tracks t
left join lyric_features lf on t.id = lf.track_id
"""
con.execute(sql)

Unfortunately, there are some nulls in the features table.

In [62]:
con.query("summarize features").show()

┌─────────────────────┬─────────────┬──────────────────────┬───┬──────────────────────┬────────┬─────────────────┐
│     column_name     │ column_type │         min          │ … │         q75          │ count  │ null_percentage │
│       varchar       │   varchar   │       varchar        │   │       varchar        │ int64  │     varchar     │
├─────────────────────┼─────────────┼──────────────────────┼───┼──────────────────────┼────────┼─────────────────┤
│ id                  │ VARCHAR     │ 000RDCYioLteXcutOj…  │ … │ NULL                 │ 101939 │ 0.0%            │
│ acousticness        │ DOUBLE      │ 0.0                  │ … │ 0.6445729934024428   │ 101939 │ 0.0%            │
│ danceability        │ DOUBLE      │ 0.0                  │ … │ 0.7144691054924385   │ 101939 │ 0.0%            │
│ duration_ms         │ DOUBLE      │ 1155.0               │ … │ 261118.58891140617   │ 101939 │ 0.0%            │
│ energy              │ DOUBLE      │ 0.0                  │ … │ 0.7975791679002

In [63]:
# Replace NULLs and -1 with 0
cols = ["mean_syllables_word", "mean_words_sentence", "n_sentences", 
        "n_words", "sentence_similarity", "vocabulary_wealth"]

for col in cols:
    # Shouldn't be done like this (sql injection)
    # but prepared statements currently are buggy in DuckDB
    sql = f"update features set {col} = 0 where {col} is null or {col} = -1"
    con.execute(sql)

In [64]:
con.query("summarize features").df()


,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,000RDCYioLteXcutOjeweY,7zzpno7uAqkAzWZDQuGEFA,101691,NaN,NaN,NaN,NaN,NaN,101939,0.0%
1,acousticness,DOUBLE,0.0,0.996,4628,0.3521235614982393,0.33485523897742075,0.04095274308893161,0.23793911430083223,0.6445729934024428,101939,0.0%
2,danceability,DOUBLE,0.0,0.989,1199,0.5860150295765181,0.1777242568588393,0.4797984757491416,0.6103442332752973,0.7144691054924385,101939,0.0%
3,duration_ms,DOUBLE,1155.0,5505831.0,58735,246770.80330393667,190430.2951407744,183983.87055703002,216896.85181616616,261118.58891140617,101939,0.0%
4,energy,DOUBLE,0.0,1.0,2353,0.5864791632319333,0.2601704025337216,0.4114203187924769,0.6284025085427688,0.7975791679002222,101939,0.0%
5,instrumentalness,DOUBLE,0.0,1.0,5341,0.14877590260086823,0.30402356366384803,0.0,3.870627597059107e-05,0.035847103131927904,101939,0.0%
6,liveness,DOUBLE,0.0,0.999,1715,0.19764030645778236,0.1753908592406057,0.09556367602621384,0.12378789284047818,0.2415826949002584,101939,0.0%
7,loudness,DOUBLE,-60.0,2.719,22382,-9.462719675492323,6.198508047679073,-11.155922531040767,-7.601996288983651,-5.513622588122102,101939,0.0%
8,mode,DOUBLE,0.0,1.0,2,0.6181539940552684,0.485841485682643,0.0,1.0,1.0,101939,0.0%
9,popularity,DOUBLE,0.0,97.0,98,39.78231098990573,16.790769499551416,28.80655572346239,40.37086281099423,51.526063473532396,101939,0.0%


## Feature Engineering

In [110]:
df = con.query("select * from features").df()
#df.set_index("id", inplace=True)

In [118]:
df.columns

Index(['id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode', 'popularity',
       'speechiness', 'tempo', 'valence', 'mean_syllables_word',
       'mean_words_sentence', 'n_sentences', 'n_words', 'sentence_similarity',
       'vocabulary_wealth'],
      dtype='object')

In [109]:
df.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,valence,mean_syllables_word,mean_words_sentence,n_sentences,n_words,sentence_similarity,vocabulary_wealth
count,101939.000000,101939.000000,1.019390e+05,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.000000,101939.00000,101939.000000,101939.000000,101939.000000,101939.000000
mean,0.352124,0.586015,2.467708e+05,0.586479,0.148776,0.197640,-9.462720,0.618154,39.782311,0.128841,118.358527,0.482813,1.134581,3.49666,40.159066,295.339821,0.045892,0.434317
std,0.334855,0.177724,1.904303e+05,0.260170,0.304024,0.175391,6.198508,0.485841,16.790769,0.203324,30.224074,0.261690,0.685974,9.27639,44.628888,844.870996,0.061666,0.254495
min,0.000000,0.000000,1.155000e+03,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.040700,0.480000,1.840000e+05,0.411000,0.000000,0.095600,-11.149000,0.000000,29.000000,0.036400,95.973000,0.271000,1.100000,2.20000,12.000000,74.000000,0.000000,0.330000
50%,0.238000,0.610000,2.168930e+05,0.629000,0.000037,0.124000,-7.599000,1.000000,41.000000,0.050600,118.067000,0.477000,1.250000,3.20000,39.000000,224.000000,0.031710,0.520000
75%,0.645000,0.714000,2.610550e+05,0.798000,0.034400,0.241000,-5.509000,1.000000,52.000000,0.104000,136.045000,0.693000,1.480000,3.93000,58.000000,345.000000,0.062987,0.620000
max,0.996000,0.989000,5.505831e+06,1.000000,1.000000,0.999000,2.719000,1.000000,97.000000,0.969000,244.035000,0.993000,4.000000,819.20000,2519.000000,39111.000000,0.964286,0.780000


In [119]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(df.loc[:, df.columns != "id"])

In [120]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10)
knn.fit(X)

NearestNeighbors(n_neighbors=10)

In [121]:
# lookup ids
track_name = "believe"
sql = f"""
    select *
    from lookup
    where regexp_matches(lower(track_name), '{track_name}')
    limit 10
"""
lookup_df = con.query(sql).df()
lookup_df

,id,track_name,artist_name,album_name,preview_url,track_href,analysis_url
0,03EdWn1FnvNuO2XOokZH2e,She Believes in the Devil,SEND MEDICINE,She Believes in the Devil,https://p.scdn.co/mp3-preview/b1b1e2849bfa75c0080413e206151602960e8cfa?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/03EdWn1FnvNuO2XOokZH2e,https://api.spotify.com/v1/audio-analysis/03EdWn1FnvNuO2XOokZH2e
1,2VvTbmiL4RJ4tlwCM1Xgvq,Make Believe,Astronaut Husband,Supergiant Blues / Drip,https://p.scdn.co/mp3-preview/fed766ef8537d52b34840eb3043494aae4a35368?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/2VvTbmiL4RJ4tlwCM1Xgvq,https://api.spotify.com/v1/audio-analysis/2VvTbmiL4RJ4tlwCM1Xgvq
2,0dhSRBTVs4kBQ9nyVFqxZH,Believe Again,Delta Goodrem,Delta,https://p.scdn.co/mp3-preview/40241a077bc887453e911c092b6483e5d3ed9424?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/0dhSRBTVs4kBQ9nyVFqxZH,https://api.spotify.com/v1/audio-analysis/0dhSRBTVs4kBQ9nyVFqxZH
3,6vyQ16BaJD028xZbJY5bUO,Would You Believe In Me,Various Artists,The Masters Series: Rare Groove,https://p.scdn.co/mp3-preview/8598a27c04467ce1766b00b671f6b7fbbf7e4733?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/6vyQ16BaJD028xZbJY5bUO,https://api.spotify.com/v1/audio-analysis/6vyQ16BaJD028xZbJY5bUO
4,6JpqZqiJrWCgSOusAARV2Q,Believer / Pretender,Sons Of Bill,Believer / Pretender,https://p.scdn.co/mp3-preview/0e839608c7b2d3b5287baf3ab05bf8df2b326dd7?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/6JpqZqiJrWCgSOusAARV2Q,https://api.spotify.com/v1/audio-analysis/6JpqZqiJrWCgSOusAARV2Q
5,51CGXCa8XqEyCiRlDIUiq1,I Believe,Third Day,Wire,https://p.scdn.co/mp3-preview/7a44c75d65b9fb0018dfbe58a04f8e388ea5cf56?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/51CGXCa8XqEyCiRlDIUiq1,https://api.spotify.com/v1/audio-analysis/51CGXCa8XqEyCiRlDIUiq1
6,1ty3N197je3o1dN0F6Fu1G,Believe,Madilyn Bailey,Believe,https://p.scdn.co/mp3-preview/0130614f636beea2f0cf8e30e08d8bddd88a06fd?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/1ty3N197je3o1dN0F6Fu1G,https://api.spotify.com/v1/audio-analysis/1ty3N197je3o1dN0F6Fu1G
7,6XpG30QPzV1wkuUhUMw64l,I Still Believe (Great Design),The Call,Reconciled,https://p.scdn.co/mp3-preview/cff522d1ab02a595206397ed79b299b2ff8c0196?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/6XpG30QPzV1wkuUhUMw64l,https://api.spotify.com/v1/audio-analysis/6XpG30QPzV1wkuUhUMw64l
8,6tAAuNXlwzUlRAiEicoGCx,Enough to Believe - Eagles & Butterflies Remix,Bob Moses,Enough to Believe (Remixes),https://p.scdn.co/mp3-preview/5493afcdd6044837a1c8be91122b277790e01ada?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/6tAAuNXlwzUlRAiEicoGCx,https://api.spotify.com/v1/audio-analysis/6tAAuNXlwzUlRAiEicoGCx
9,6BVZyHVBeNkPaLF2e7G0c1,Enough to Believe - Jamie Jones Remix,Bob Moses,Enough to Believe (Jamie Jones Remix),https://p.scdn.co/mp3-preview/13e16abc25f756585ca253e4f5d8555b81cdfcbb?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/6BVZyHVBeNkPaLF2e7G0c1,https://api.spotify.com/v1/audio-analysis/6BVZyHVBeNkPaLF2e7G0c1


In [ ]:
# Make test data
#track_id = lookup_df["id"][0:1]
#X_test = scaler.transform(df.loc[track_id])

In [126]:
# Make test data
df_test = df.query("id == '6tAAuNXlwzUlRAiEicoGCx'").loc[:, df.columns != "id"]
X_test = scaler.transform(df_test)

In [127]:
# Predict
distances, indices = knn.kneighbors(X_test)

In [128]:
distances

array([[0.        , 0.12294053, 0.16283115, 0.16641119, 0.17837178,
        0.17984785, 0.19032366, 0.19044526, 0.19306232, 0.19408442]])

In [132]:
indices[0].tolist()

[9670, 45416, 66333, 66991, 1721, 10375, 22444, 79044, 74033, 66804]

In [150]:
df.loc[indices[0].tolist(), :].id.to_list()

['6tAAuNXlwzUlRAiEicoGCx',
 '7xLDFnimj6kxeExmTpQYd4',
 '2mjpdlZYYdAz0Sg9Z1k7dU',
 '6Bo4zbv7hfhCTtLZZnOol4',
 '7uw7m2p9NAMApIPygQ3h1H',
 '5tIhRlNkApQJoDA8zhOBUY',
 '1wYg1Ud13bBAJgKocnm73h',
 '4njxXtkcvrPS1919VgbAjG',
 '5DbEn82RVTtthtu9xvYCtb',
 '18oSjj6BaTL3NnxDkUZ5Tn']

## Clean Up

In [5]:
con.close()